# ChildBERT
- https://github.com/monologg/KoBERT-Transformers<br>
- https://github.com/SKTBrain/KoBERT<br>
- https://huggingface.co/docs/transformers/model_doc/bert

In [ ]:
##Parameter setting
setEpoch = 10
setLearningRate = 0.0001
setEpsilon = 1e-8
setBatch = 32
setMaxLength = 256
setSeed = 42
labelNumber = 2
setTry = 31

In [ ]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')
    
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

Found GPU at: /device:GPU:0
There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [ ]:
!pip install transformers
!pip install kobert_transformers
!pip install tensorflow
!pip install keras
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 6.8 MB/s 
     |████████████████████████████████| 6.6 MB 51.4 MB/s 
     |████████████████████████████████| 596 kB 72.7 MB/s 
     |████████████████████████████████| 101 kB 12.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 9.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime
import sentencepiece as spm

In [ ]:
# Mount Google Drive to this Notebook instance.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# coding=utf-8
# Copyright 2018 Google AI, Google Brain and Carnegie Mellon University Authors and the HuggingFace Inc. team and Jangwon Park
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
""" Tokenization classes for KoBert model."""


import logging
import os
import unicodedata
from shutil import copyfile

from transformers import PreTrainedTokenizer


logger = logging.getLogger(__name__)

VOCAB_FILES_NAMES = {"vocab_file": "tokenizer_78b3253a26.model",
                     "vocab_txt": "vocab.txt"}

PRETRAINED_VOCAB_FILES_MAP = {
    "vocab_file": {
        "monologg/kobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert/tokenizer_78b3253a26.model",
        "monologg/kobert-lm": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert-lm/tokenizer_78b3253a26.model",
        "monologg/distilkobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/distilkobert/tokenizer_78b3253a26.model"
    },
    "vocab_txt": {
        "monologg/kobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert/vocab.txt",
        "monologg/kobert-lm": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert-lm/vocab.txt",
        "monologg/distilkobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/distilkobert/vocab.txt"
    }
}

PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES = {
    "monologg/kobert": 512,
    "monologg/kobert-lm": 512,
    "monologg/distilkobert": 512
}

PRETRAINED_INIT_CONFIGURATION = {
    "monologg/kobert": {"do_lower_case": False},
    "monologg/kobert-lm": {"do_lower_case": False},
    "monologg/distilkobert": {"do_lower_case": False}
}

SPIECE_UNDERLINE = u'▁'


class KoBertTokenizer(PreTrainedTokenizer):
    """
        SentencePiece based tokenizer. Peculiarities:
            - requires `SentencePiece <https://github.com/google/sentencepiece>`_
    """
    vocab_files_names = VOCAB_FILES_NAMES
    pretrained_vocab_files_map = PRETRAINED_VOCAB_FILES_MAP
    pretrained_init_configuration = PRETRAINED_INIT_CONFIGURATION
    max_model_input_sizes = PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES

    def __init__(
            self,
            vocab_file,
            vocab_txt,
            do_lower_case=False,
            remove_space=True,
            keep_accents=False,
            unk_token="[UNK]",
            sep_token="[SEP]",
            pad_token="[PAD]",
            cls_token="[CLS]",
            mask_token="[MASK]",
            **kwargs):
        super().__init__(
            unk_token=unk_token,
            sep_token=sep_token,
            pad_token=pad_token,
            cls_token=cls_token,
            mask_token=mask_token,
            **kwargs
        )

        # Build vocab
        self.token2idx = dict()
        self.idx2token = []
        with open(vocab_txt, 'r', encoding='utf-8') as f:
            for idx, token in enumerate(f):
                token = token.strip()
                self.token2idx[token] = idx
                self.idx2token.append(token)

        try:
            import sentencepiece as spm
        except ImportError:
            logger.warning("You need to install SentencePiece to use KoBertTokenizer: https://github.com/google/sentencepiece"
                           "pip install sentencepiece")

        self.do_lower_case = do_lower_case
        self.remove_space = remove_space
        self.keep_accents = keep_accents
        self.vocab_file = vocab_file
        self.vocab_txt = vocab_txt

        self.sp_model = spm.SentencePieceProcessor()
        self.sp_model.Load(vocab_file)

    @property
    def vocab_size(self):
        return len(self.idx2token)

    def get_vocab(self):
        return dict(self.token2idx, **self.added_tokens_encoder)

    def __getstate__(self):
        state = self.__dict__.copy()
        state["sp_model"] = None
        return state

    def __setstate__(self, d):
        self.__dict__ = d
        try:
            import sentencepiece as spm
        except ImportError:
            logger.warning("You need to install SentencePiece to use KoBertTokenizer: https://github.com/google/sentencepiece"
                           "pip install sentencepiece")
        self.sp_model = spm.SentencePieceProcessor()
        self.sp_model.Load(self.vocab_file)

    def preprocess_text(self, inputs):
        if self.remove_space:
            outputs = " ".join(inputs.strip().split())
        else:
            outputs = inputs
        outputs = outputs.replace("``", '"').replace("''", '"')

        if not self.keep_accents:
            outputs = unicodedata.normalize('NFKD', outputs)
            outputs = "".join([c for c in outputs if not unicodedata.combining(c)])
        if self.do_lower_case:
            outputs = outputs.lower()

        return outputs

    def _tokenize(self, text, return_unicode=True, sample=False):
        """ Tokenize a string. """
        text = self.preprocess_text(text)

        if not sample:
            pieces = self.sp_model.EncodeAsPieces(text)
        else:
            pieces = self.sp_model.SampleEncodeAsPieces(text, 64, 0.1)
        new_pieces = []
        for piece in pieces:
            if len(piece) > 1 and piece[-1] == str(",") and piece[-2].isdigit():
                cur_pieces = self.sp_model.EncodeAsPieces(piece[:-1].replace(SPIECE_UNDERLINE, ""))
                if piece[0] != SPIECE_UNDERLINE and cur_pieces[0][0] == SPIECE_UNDERLINE:
                    if len(cur_pieces[0]) == 1:
                        cur_pieces = cur_pieces[1:]
                    else:
                        cur_pieces[0] = cur_pieces[0][1:]
                cur_pieces.append(piece[-1])
                new_pieces.extend(cur_pieces)
            else:
                new_pieces.append(piece)

        return new_pieces

    def _convert_token_to_id(self, token):
        """ Converts a token (str/unicode) in an id using the vocab. """
        return self.token2idx.get(token, self.token2idx[self.unk_token])

    def _convert_id_to_token(self, index, return_unicode=True):
        """Converts an index (integer) in a token (string/unicode) using the vocab."""
        return self.idx2token[index]

    def convert_tokens_to_string(self, tokens):
        """Converts a sequence of tokens (strings for sub-words) in a single string."""
        out_string = "".join(tokens).replace(SPIECE_UNDERLINE, " ").strip()
        return out_string

    def build_inputs_with_special_tokens(self, token_ids_0, token_ids_1=None):
        """
        Build model inputs from a sequence or a pair of sequence for sequence classification tasks
        by concatenating and adding special tokens.
        A KoBERT sequence has the following format:
            single sequence: [CLS] X [SEP]
            pair of sequences: [CLS] A [SEP] B [SEP]
        """
        if token_ids_1 is None:
            return [self.cls_token_id] + token_ids_0 + [self.sep_token_id]
        cls = [self.cls_token_id]
        sep = [self.sep_token_id]
        return cls + token_ids_0 + sep + token_ids_1 + sep

    def get_special_tokens_mask(self, token_ids_0, token_ids_1=None, already_has_special_tokens=False):
        """
        Retrieves sequence ids from a token list that has no special tokens added. This method is called when adding
        special tokens using the tokenizer ``prepare_for_model`` or ``encode_plus`` methods.
        Args:
            token_ids_0: list of ids (must not contain special tokens)
            token_ids_1: Optional list of ids (must not contain special tokens), necessary when fetching sequence ids
                for sequence pairs
            already_has_special_tokens: (default False) Set to True if the token list is already formated with
                special tokens for the model
        Returns:
            A list of integers in the range [0, 1]: 0 for a special token, 1 for a sequence token.
        """

        if already_has_special_tokens:
            if token_ids_1 is not None:
                raise ValueError(
                    "You should not supply a second sequence if the provided sequence of "
                    "ids is already formated with special tokens for the model."
                )
            return list(map(lambda x: 1 if x in [self.sep_token_id, self.cls_token_id] else 0, token_ids_0))

        if token_ids_1 is not None:
            return [1] + ([0] * len(token_ids_0)) + [1] + ([0] * len(token_ids_1)) + [1]
        return [1] + ([0] * len(token_ids_0)) + [1]

    def create_token_type_ids_from_sequences(self, token_ids_0, token_ids_1=None):
        """
        Creates a mask from the two sequences passed to be used in a sequence-pair classification task.
        A KoBERT sequence pair mask has the following format:
        0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
        | first sequence    | second sequence
        if token_ids_1 is None, only returns the first portion of the mask (0's).
        """
        sep = [self.sep_token_id]
        cls = [self.cls_token_id]
        if token_ids_1 is None:
            return len(cls + token_ids_0 + sep) * [0]
        return len(cls + token_ids_0 + sep) * [0] + len(token_ids_1 + sep) * [1]

    def save_vocabulary(self, save_directory):
        """ Save the sentencepiece vocabulary (copy original file) and special tokens file
            to a directory.
        """
        if not os.path.isdir(save_directory):
            logger.error("Vocabulary path ({}) should be a directory".format(save_directory))
            return

        # 1. Save sentencepiece model
        out_vocab_model = os.path.join(save_directory, VOCAB_FILES_NAMES["vocab_file"])

        if os.path.abspath(self.vocab_file) != os.path.abspath(out_vocab_model):
            copyfile(self.vocab_file, out_vocab_model)

        # 2. Save vocab.txt
        index = 0
        out_vocab_txt = os.path.join(save_directory, VOCAB_FILES_NAMES["vocab_txt"])
        with open(out_vocab_txt, "w", encoding="utf-8") as writer:
            for token, token_index in sorted(self.token2idx.items(), key=lambda kv: kv[1]):
                if index != token_index:
                    logger.warning(
                        "Saving vocabulary to {}: vocabulary indices are not consecutive."
                        " Please check that the vocabulary is not corrupted!".format(out_vocab_txt)
                    )
                    index = token_index
                writer.write(token + "\n")
                index += 1

        return out_vocab_model, out_vocab_txt

In [ ]:
tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')
print(tokenizer.tokenize("사자가 햄을 맛있게 먹어요"))
print(tokenizer("사자가 햄을 맛있게 먹어요"))
print("KoBERT: ",len(tokenizer.get_vocab()))

Downloading:   0%|          | 0.00/363k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/76.0k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


['▁사', '자가', '▁', '햄', '을', '▁맛', '있', '게', '▁먹', '어요']
{'input_ids': [2, 2573, 7148, 517, 7862, 7088, 1967, 7141, 5400, 2010, 6857, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
KoBERT:  8002


In [ ]:
tokenizerSU = BertTokenizer.from_pretrained('snunlp/KR-BERT-char16424')
print(tokenizerSU.tokenize("사자가 햄을 맛있게 먹어요"))
print(tokenizerSU("사자가 햄을 맛있게 먹어요"))
print("KrBERT: ",len(tokenizer.get_vocab()))


Downloading:   0%|          | 0.00/101k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/389 [00:00<?, ?B/s]

['사', '##자가', '햄', '##을', '맛있', '##게', '먹', '##어요']
{'input_ids': [2, 138, 870, 14001, 6, 7072, 101, 1045, 2324, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
KrBERT:  8002


In [ ]:
for currentTry in range(16,setTry):
  import pandas as pd

  fileDir = "drive/My Drive/2022년/문성민 박사/딥러닝/BiClassification/Data/Binary/trainBi.csv"

  fr = open(fileDir, 'r')
  contents = fr.readlines()
  fr.close()

  train = pd.DataFrame(columns=('Label', 'Sentence'))
  i = 0
  label = ""
  sentence = ""
  for content in contents:
      if i == 0:
          pass
      else:
          infos = content.split(",")
          label = int(infos[2])
          sentence = infos[3].replace("\n", "")
          train.loc[i] = [label, sentence]
      i = i + 1


  train['Sentence'] = train['Sentence'].str.replace(r'[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》\\n\t]+', " ",regex=True)
  train['Sentence'] = train['Sentence'].str.replace(r'\t+', " ", regex=True)
  train['Sentence'] = train['Sentence'].str.replace(r'[\\n]+', " ", regex=True)

  # 리뷰 문장 추출
  sentences = train['Sentence']

  # BERT의 입력 형식에 맞게 변환
  sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]

  # 라벨 추출
  labels = train['Label'].values
  labels_re = []
  for label in labels:
    labels_re.append(label)
  labels = labels_re

  # # BERT의 토크나이저로 문장을 토큰으로 분리
  # from kobert_transformers import get_tokenizer
  # tokenizer = get_tokenizer()
  # tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
  # BERT의 토크나이저로 문장을 토큰으로 분리
  tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

  print("Before patching: ",len(tokenizer.get_vocab()))

  # CHILDES데이터 사용하여 사전학습 모델 튜닝
  entireCorpus = "drive/My Drive/2022년/문성민 박사/딥러닝/BiClassification/Data/Binary/CHILDES/CHILDES_all_spellchecked_corrected.txt"
  entireFr = open(entireCorpus, 'r')
  entireContents = entireFr.readlines()
  entireFr.close()

  import re

  entireSet = set()
  entireContentList = list()
  #공백을 기준으로 단어를 생성하고 추가하기
  for entireContent in entireContents:
      entireContent = re.sub('[^가-힣]', ' ', entireContent)
      entireContent = re.sub('[\s]+', ' ', entireContent)
      entireContentList.append(entireContent.replace("\n",""))

      entireContentSplit = entireContent.split(" ")
      for each in entireContentSplit:
        if each != "":
          entireSet.add(each)

  # print(len(entireSet)) #37898
  wordDic = {}
  for eachWord in entireSet:
    wordDic[eachWord] = 0

  for contentEach in entireContentList:
    contentEachSplit = contentEach.split(" ")
    for each in contentEachSplit:
      if each != "":
        wordDic[each] = wordDic[each] + 1

  wordDicSorted = dict(sorted(wordDic.items(), key=lambda x: x[1], reverse=True))

  #사전에 추가하기
  countNum = 1
  for key, value in wordDicSorted.items():
    if value > 1:
      countNum = countNum + 1
      tokenizer.add_tokens([key])

  # print(countNum) #3630

  print("After patching: ",len(tokenizer.get_vocab()))


  tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

  # 입력 토큰의 최대 시퀀스 길이
  MAX_LEN = setMaxLength  # 64 128 256 512

  # 토큰을 숫자 인덱스로 변환
  input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

  # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
  input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

  print(input_ids[0])


  # 어텐션 마스크 초기화
  attention_masks = []

  # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
  # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
  for seq in input_ids:
      seq_mask = [float(i>0) for i in seq]
      attention_masks.append(seq_mask)

  print(attention_masks[0])
  print(labels)
  print(input_ids)


  # 훈련셋과 검증셋으로 분리
  train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
                                                                                      labels, 
                                                                                      random_state=2018, 
                                                                                      test_size=0.05)

  # 어텐션 마스크를 훈련셋과 검증셋으로 분리
  train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                        input_ids,
                                                        random_state=2018, 
                                                        test_size=0.05)

  # 데이터를 파이토치의 텐서로 변환
  train_inputs = torch.tensor(train_inputs)
  train_labels = torch.tensor(train_labels)
  train_masks = torch.tensor(train_masks)
  validation_inputs = torch.tensor(validation_inputs)
  validation_labels = torch.tensor(validation_labels)
  validation_masks = torch.tensor(validation_masks)

  # 배치 사이즈
  batch_size = setBatch

  # 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
  # 학습시 배치 사이즈 만큼 데이터를 가져옴
  train_data = TensorDataset(train_inputs, train_masks, train_labels)
  train_sampler = RandomSampler(train_data)
  train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

  validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
  validation_sampler = SequentialSampler(validation_data)
  validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

  # 분류를 위한 BERT 모델 생성
  model = BertForSequenceClassification.from_pretrained("monologg/kobert", num_labels=labelNumber)
  model.cuda()

  model.resize_token_embeddings(len(tokenizer.get_vocab()))

  # 정확도 계산 함수
  def flat_accuracy(preds, labels):
      
      pred_flat = np.argmax(preds, axis=1).flatten()
      labels_flat = labels.flatten()

      return np.sum(pred_flat == labels_flat) / len(labels_flat)

  # 시간 표시 함수
  def format_time(elapsed):

      # 반올림
      elapsed_rounded = int(round((elapsed)))
      
      # hh:mm:ss으로 형태 변경
      return str(datetime.timedelta(seconds=elapsed_rounded))

  # 입력 데이터 변환
  def convert_input_data(sentences):

      # BERT의 토크나이저로 문장을 토큰으로 분리
      tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

      # 입력 토큰의 최대 시퀀스 길이
      MAX_LEN = 256

      # 토큰을 숫자 인덱스로 변환
      input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
      
      # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
      input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

      # 어텐션 마스크 초기화
      attention_masks = []

      # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
      # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
      for seq in input_ids:
          seq_mask = [float(i>0) for i in seq]
          attention_masks.append(seq_mask)

      # 데이터를 파이토치의 텐서로 변환
      inputs = torch.tensor(input_ids)
      masks = torch.tensor(attention_masks)

      return inputs, masks

  ##############################결과구문으로출력###################################
  def outreault(guess):
      guess = int(guess)
      outConstruction = ""
      if guess == 0:
          outConstruction = "agent-first"
      elif guess == 1:
          outConstruction = "theme-first"

      return outConstruction


  # 옵티마이저 설정
  optimizer = AdamW(model.parameters(),
                    lr = setLearningRate, # 학습률
                    eps = setEpsilon # 0으로 나누는 것을 방지하기 위한 epsilon 값
                  )

  # 에폭수
  epochs = setEpoch

  # 총 훈련 스텝 : 배치반복 횟수 * 에폭
  total_steps = len(train_dataloader) * epochs

  # 학습률을 조금씩 감소시키는 스케줄러 생성
  scheduler = get_linear_schedule_with_warmup(optimizer, 
                                              num_warmup_steps = 0,
                                              num_training_steps = total_steps)


  #학습
  # 재현을 위해 랜덤시드 고정
  seed_val = setSeed
  random.seed(seed_val)
  np.random.seed(seed_val)
  torch.manual_seed(seed_val)
  torch.cuda.manual_seed_all(seed_val)

  # 그래디언트 초기화
  model.zero_grad()


  outDir = "drive/My Drive/2022년/문성민 박사/딥러닝/BiClassification/Data/Binary_2023/OutputBERT/bertBi_All_T"+str(currentTry)+".csv"
  f = open(outDir, 'w')
  f.write("epoch,sentence,originalLabel,predictedLabel,predictedConstruction,result"+"\n")

  # 에폭만큼 반복
  for epoch_i in range(0, epochs):
      
      # ========================================
      #               Training
      # ========================================
      
      print("")
      print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
      print('Training...')

      # 시작 시간 설정
      t0 = time.time()

      # 로스 초기화
      total_loss = 0

      # 훈련모드로 변경
      model.train()
          
      # 데이터로더에서 배치만큼 반복하여 가져옴
      for step, batch in enumerate(train_dataloader):
          # 경과 정보 표시
          if step % 500 == 0 and not step == 0:
              elapsed = format_time(time.time() - t0)
              print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

          # 배치를 GPU에 넣음
          batch = tuple(t.to(device) for t in batch)
          
          # 배치에서 데이터 추출
          b_input_ids, b_input_mask, b_labels = batch

          # Forward 수행                
          outputs = model(b_input_ids, 
                          token_type_ids=None, 
                          attention_mask=b_input_mask, 
                          labels=b_labels)
          
          # 로스 구함
          loss = outputs[0]

          # 총 로스 계산
          total_loss += loss.item()

          # Backward 수행으로 그래디언트 계산
          loss.backward()

          # 그래디언트 클리핑
          torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

          # 그래디언트를 통해 가중치 파라미터 업데이트
          optimizer.step()

          # 스케줄러로 학습률 감소
          scheduler.step()

          # 그래디언트 초기화
          model.zero_grad()

      # 평균 로스 계산
      avg_train_loss = total_loss / len(train_dataloader)            

      print("")
      print("  Average training loss: {0:.2f}".format(avg_train_loss))
      print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
          
      # ========================================
      #               Validation
      # ========================================

      print("")
      print("Running Validation...")

      #시작 시간 설정
      t0 = time.time()

      # 평가모드로 변경
      model.eval()

      # 변수 초기화
      eval_loss, eval_accuracy = 0, 0
      nb_eval_steps, nb_eval_examples = 0, 0

      # 데이터로더에서 배치만큼 반복하여 가져옴
      for batch in validation_dataloader:
          # 배치를 GPU에 넣음
          batch = tuple(t.to(device) for t in batch)
          
          # 배치에서 데이터 추출
          b_input_ids, b_input_mask, b_labels = batch
          
          # 그래디언트 계산 안함
          with torch.no_grad():     
              # Forward 수행
              outputs = model(b_input_ids, 
                              token_type_ids=None, 
                              attention_mask=b_input_mask)
          
          # 로스 구함
          logits = outputs[0]

          # CPU로 데이터 이동
          logits = logits.detach().cpu().numpy()
          label_ids = b_labels.to('cpu').numpy()
          
          # 출력 로짓과 라벨을 비교하여 정확도 계산
          tmp_eval_accuracy = flat_accuracy(logits, label_ids)
          eval_accuracy += tmp_eval_accuracy
          nb_eval_steps += 1

          

      print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
      print("  Validation took: {:}".format(format_time(time.time() - t0)))


      if (epoch_i+1) == 1 or (epoch_i+1) == 2 or (epoch_i+1) == 3 or (epoch_i+1) == 4 or (epoch_i+1) == 5 or (epoch_i+1) == 6 or (epoch_i+1) == 7 or (epoch_i+1) == 8 or (epoch_i+1) == 9 or (epoch_i+1) == 10 :#or (epoch_i+1) == 20 or (epoch_i+1) == 30: #or (epoch_i+1) == 40 or (epoch_i+1) == 50:
        # ###############################전체 모델 성능 평가########################################

        # 문장 테스트
        def test_sentences(sentences):
            # 평가모드로 변경
            model.eval()

            # 문장을 입력 데이터로 변환
            inputs, masks = convert_input_data(sentences)

            # 데이터를 GPU에 넣음
            b_input_ids = inputs.to(device)
            b_input_mask = masks.to(device)
                    
            # 그래디언트 계산 안함
            with torch.no_grad():     
                # Forward 수행
                outputs = model(b_input_ids, 
                                token_type_ids=None, 
                                attention_mask=b_input_mask)

            # 로스 구함
            logits = outputs[0]

            # CPU로 데이터 이동
            logits = logits.detach().cpu().numpy()

            return logits

        import pandas as pd

        testFileDir = fileDir = "drive/My Drive/2022년/문성민 박사/딥러닝/BiClassification/Data/Binary/testBi_All.csv" #"drive/My Drive/NLP/GH/BUCLD/Data/Binary/testBi_BUCLD.csv"
        testFr = open(testFileDir, 'r')
        testContents = testFr.readlines()
        testFr.close()

        test = pd.DataFrame(columns=('Label1', 'Label2','Sentence'))
        i = 0
        for content in testContents:
            if i == 0:
                pass
            else:
                infos = content.split(",")
                label1 = int(infos[0])
                label2 = int(infos[1])
                sentence = infos[2].replace("\n", "")
                test.loc[i] = [label1, label2, sentence]
            i = i + 1

        test['Sentence'] = test['Sentence'].str.replace(r'[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》\\n\t]+', " ", regex=True)
        test['Sentence'] = test['Sentence'].str.replace(r'\t+', " ", regex=True)
        test['Sentence'] = test['Sentence'].str.replace(r'[\\n]+', " ", regex=True)

        # 리뷰 문장 추출
        testSentences = test['Sentence']

        totalNum = 0
        correctNum = 0
        for each in range(0, len(testSentences)):
            print(test['Label1'][each + 1], test['Label2'][each + 1], test['Sentence'][each + 1])
            logits = test_sentences([test['Sentence'][each + 1]])
            guess = str(np.argmax(logits))
            if guess == str(test['Label1'][each + 1]) or guess == str(test['Label2'][each + 1]):
                print("input: ", test['Sentence'][each + 1], ", predict: ", guess, "(O)")
                f.write(str(epoch_i+1) + "," + test['Sentence'][each + 1] + "," + str(test['Label1'][each + 1]) + "or" + str(test['Label2'][each + 1]) + "," + guess + "," + outreault(guess)+ ",1" + "\n")
                correctNum = correctNum + 1
            else:
                f.write(str(epoch_i+1) + "," + test['Sentence'][each + 1] + "," + str(test['Label1'][each + 1]) + "or" + str(test['Label2'][each + 1]) + "," + guess + "," + outreault(guess) + ",0" + "\n")       
                print("input: ", test['Sentence'][each + 1], ", predict: ", guess, "(X)")
            totalNum = totalNum + 1

        print("totalNum: ", totalNum, " correctNum: ", correctNum, " accuracy: ", (correctNum/totalNum))

  f.close()



      

  print("")
  print("Training complete!")


Downloading:   0%|          | 0.00/363k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/76.0k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


...


======== Epoch 1 / 10 ========
Training...

  Average training loss: 0.68
  Training epcoh took: 0:02:32

Running Validation...
  Accuracy: 0.72
  Validation took: 0:00:03
0 0 호랑이가 코끼리를 그려요
input:  호랑이가 코끼리를 그려요 , predict:  1 (X)
0 0 원숭이가 다람쥐를 던져요
input:  원숭이가 다람쥐를 던져요 , predict:  1 (X)
0 0 꿀벌이 다람쥐를 칠해요
input:  꿀벌이 다람쥐를 칠해요 , predict:  1 (X)
0 0 꿀벌이 나비를 안아요
input:  꿀벌이 나비를 안아요 , predict:  0 (O)
0 0 다람쥐가 호랑이를 밀어요
input:  다람쥐가 호랑이를 밀어요 , predict:  0 (O)
0 0 개구리가 원숭이를 들어요
input:  개구리가 원숭이를 들어요 , predict:  0 (O)
0 0 코끼리가 호랑이 그려요
input:  코끼리가 호랑이 그려요 , predict:  1 (X)
0 0 다람쥐가 원숭이 던져요
input:  다람쥐가 원숭이 던져요 , predict:  1 (X)
0 0 다람쥐가 꿀벌 칠해요
input:  다람쥐가 꿀벌 칠해요 , predict:  1 (X)
0 0 나비가 꿀벌 안아요
input:  나비가 꿀벌 안아요 , predict:  0 (O)
0 0 호랑이가 다람쥐 밀어요
input:  호랑이가 다람쥐 밀어요 , predict:  0 (O)
0 0 원숭이가 개구리 들어요
input:  원숭이가 개구리 들어요 , predict:  0 (O)
0 0 코끼리 호랑이를 그려요
input:  코끼리 호랑이를 그려요 , predict:  1 (X)
0 0 다람쥐 원숭이를 던져요
input:  다람쥐 원숭이를 던져요 , predict:  0 (O)
0 0 다람쥐 꿀벌을 칠해요
input:  다람쥐 꿀벌을 칠해요 , predi

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


Before patching:  8002
After patching:  24857
[    2 11533  8872 11210  9799 15717     3     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



======== Epoch 1 / 10 ========
Training...

  Average training loss: 0.68
  Training epcoh took: 0:02:32

Running Validation...
  Accuracy: 0.72
  Validation took: 0:00:03
0 0 호랑이가 코끼리를 그려요
input:  호랑이가 코끼리를 그려요 , predict:  1 (X)
0 0 원숭이가 다람쥐를 던져요
input:  원숭이가 다람쥐를 던져요 , predict:  1 (X)
0 0 꿀벌이 다람쥐를 칠해요
input:  꿀벌이 다람쥐를 칠해요 , predict:  1 (X)
0 0 꿀벌이 나비를 안아요
input:  꿀벌이 나비를 안아요 , predict:  0 (O)
0 0 다람쥐가 호랑이를 밀어요
input:  다람쥐가 호랑이를 밀어요 , predict:  0 (O)
0 0 개구리가 원숭이를 들어요
input:  개구리가 원숭이를 들어요 , predict:  0 (O)
0 0 코끼리가 호랑이 그려요
input:  코끼리가 호랑이 그려요 , predict:  1 (X)
0 0 다람쥐가 원숭이 던져요
input:  다람쥐가 원숭이 던져요 , predict:  1 (X)
0 0 다람쥐가 꿀벌 칠해요
input:  다람쥐가 꿀벌 칠해요 , predict:  1 (X)
0 0 나비가 꿀벌 안아요
input:  나비가 꿀벌 안아요 , predict:  0 (O)
0 0 호랑이가 다람쥐 밀어요
input:  호랑이가 다람쥐 밀어요 , predict:  0 (O)
0 0 원숭이가 개구리 들어요
input:  원숭이가 개구리 들어요 , predict:  0 (O)
0 0 코끼리 호랑이를 그려요
input:  코끼리 호랑이를 그려요 , predict:  1 (X)
0 0 다람쥐 원숭이를 던져요
input:  다람쥐 원숭이를 던져요 , predict:  0 (O)
0 0 다람쥐 꿀벌을 칠해요
input:  다람쥐 꿀벌을 칠해요 , predi